# Southeast Arizona Weather Analysis

This is a report on the historical analysis of weather patterns in an area that approximately overlaps the area of the southeast Arizona, close to city Tucson.

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

<p><img alt="map.png" src="hw5_figures/map.png" style="height:300px; width:500px" /></p>

We focused on six measurements:
* **TMIN, TMAX:** the daily minimum and maximum temperature.
* **TOBS:** The average temperature for each day.
* **PRCP:** Daily Percipitation (in mm)
* **SNOW:** Daily snowfall (in mm)
* **SNWD:** The depth of accumulated snow.

## Sanity-check: comparison with outside sources

<p>We start by comparing some of the general statistics with graphs that we obtained from a site called <a href="http://www.usclimatedata.com/climate/boston/massachusetts/united-states/usma0046" target="_blank">US Climate Data</a> The graph below shows the daily minimum and maximum temperatures for each month, as well as the total precipitation for each month. The data is from city Tucson, Arizona, which is very close to the area of our dataset in HW5.</p>
<p>&nbsp;</p>
<p><img alt="tucson_onweb.jpg" src="hw5_figures/tucson_onweb.jpg" style="height:300px; width:450px" /></p>
<p>&nbsp;</p>

<p>We see that the min and max daily temperature agree with the ones we got from our data. In our data, the value of y axis divided by 10 is equal to the actual temperature in centigrade. For example, the highest temperature is in early July. In the plot TMAX, we see the mean TMAX in July is 37 celsius， which converted to Fahrenheit is 98.6. It is very close to the peek in the graph from US Climate Data. </p>

<P><img alt="TMIN_TMAX" src="hw5_figures/TMIN_TMAX.png"></P>

Now we compare PRCP. We can see the shape of the curve of our data is very close to the data from the website. For example, the highest precipitation is in August. From our data, we see the daily PRCP in August is about 21/10 = 2.1 mm. So in month, PRCP is 2.1*30 = 63mm per month. When converted to inch, it is 2.48 inch per month. And the peek in the histogram above is very close to 2.48 inch.

Besides, we can clearly see the precipitation between April and June is very low. 

<p>&nbsp;<img alt="PRCP.png" src="hw5_figures/PRCP.png" style="height:450px; width:600px" /></p>

<p>We also have the data for SNOW and SNWD. In the graph we see in summer, there is little snow in that area. And in other months of the year, the snow is also very rare because the mean is close to 0.</p>
<p><img alt="SNOW" src="hw5_figures/SNOW.png"></p>

Based on the data and graph we generated, combining with our knowledge, all these conclusions are reasonable. Arizona is rounded by desert and is very hot in summer. Also, due to local climate, precipitation is very low and almost no snow over the year.

## PCA Analysis

For each of the six measurement, we compute the percentate of the variance explained as a function of the number of eigen-vectors used.

### Percentage of variance explained
![perc1.png](hw5_figures/perc1.png)
We see that the top 5 eigen-vectors explain 47% of variance for TMIN, 60% for TOBS and 47% for TMAX.
We conclude that of the three, TOBS is best explained by the top 5 eigenvectors. This is especially true for the first eigen-vector which, by itself, explains 52% of the variance.

![perc2.png](hw5_figures/perc2.png)

The top 5 eigenvectors explain 8.5% of the variance for PRCP and 16% for SNOW. Both are low values. On the other hand the top 5 eigenvectors explain %87 of the variance for SNWD. This means that these top 5 eigenvectors capture most of the variation in the snow signals. 
It makes sense that SNWD would be less noisy than SNOW. That is because SNWD is a decaying integral of SNOW and, as such, varies less between days and between the same date on diffferent years.

Based on that we will dig deeper into the PCA analysis for snow-depth.

## Analysis of snow depth

### Mean and eigen-vectors

We choose to analyze the eigen-decomposition for snow-depth because the first 3 eigen-vectors explain 82% of the variance.

We get the mean and the top 3 eigenvectors.

We observe that the snow season is from November to the April, where the peak of the snow-depth is around January.

<p><img alt="SNWD_eigen" src="hw5_figures/SNWD_eigen.png" style="height:450px; width:600px"></p>

In the eigenvector plot, we see eig1 has the similar shape as mean but is flat compared to mean. Eig2 and eig3 oscilate between positive and negative values and they contribute the distribution of of snow depth over the year. We can interpret top 3 eigenvectors in this way:
* **eig1:** determine the tendency of SNWD, little snow between April and November.
* **eig2:** little snow in the end of December.
* **eig3:** more snow in March, little snow in Feb.


### Best reconstruction
Calculating the residual normalized norm, we can tell which one is the best reconstruction of our original data. And with the help of Eigen_decomp method, we can get the plot of best reconstruction.

<p><img alt="best_reconstruction" src="hw5_figures/best_reconstruction.png" style="height:450px; width:600px"></p>

We see the residual normalized norm after mean is near 1, but after adding top eigs, the norm value decreases rapidly. We can conclude that eig1 can reconstruct the original data very well. But of course, the best reconstruction will be the combination of mean and top 3 eigs. 

The conclusion above is illustrated by the graph directly. We see mean is very flat but the red curve is very close to the target.

### Estimating the effect of the year vs the effect of the station
To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

We get the following results:
<p><img src="hw5_figures/RMS.png" style="height:100px; width:400px"></p>

<p>We see RMS removing mean-by-year is much smaller than RMS removing mean-by-station. This means year plays a more significant role in effects on data.</p>

## Analysis of correlation between percipitation across locations


### Definition of statistical test

We want to find a statistical test for rejecting the null hypothesis that says that the rainfall in the two locations is independent.

Using the inner product is too noisy, because you multiply the rainfall on the same day in two locations and that product can be very large - leading to a large variance and poor ability to discriminate.

An alternative is to ignore the amount of rain, and just ask whether it rained in both locations. We can then compute the probability associated with the number of overlaps under the null hypothesis.

Fix two stations. We restrict our attention to the days for which we have measurements for both stations, and define the following notation:
* $m$ : the total number of days (for which we have measurements for both stations).
* $n_1$ : the number of days that it rained on station 1
* $n_2$ : the number of days that it rained on station 2
* $l$ : the number of days that it rained on both stations.

We want to calculate the probability that the number of overlap days is $l$ given $m,n_1,n_2$.

The answer is:
$$
P = {m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} /{m \choose n_1}{m \choose n_2}
$$

Where
$$
{m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} = \frac{m!}{l! (n_1-l)! (n_2-l)! (m-n_1-n_2+l)!}
$$

We use the fact that $\Gamma(n+1) = n!$ and denote $G(n) \doteq \log \Gamma(n+1)$
$$
\log P = \left[G(m) - G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) \right] - 
\left[G(m)-G(n_1)-G(m-n_1)\right] - \left[G(m)-G(n_2)-G(m-n_2)\right]
$$
Which slightly simplifies to 
$$
\log P = -G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) - G(m)+G(n_1)+G(m-n_1) +G(n_2)+G(m-n_2)
$$

The log probability scales with $m$ the length of the overlap. So to get a per-day significance we consider $
\frac{1}{m} \log P $

### Correlations matrix

<p><img src="hw5_figures/correlation_matrix.png" style="height:450px; width:450px"></p>

The matrix above shows, for each pair of stations, the normalized log probability that the overlap in rain days is random.
We see immediately the first 80 stations are highly correlated with each other.


To find more correlations we use SVD. As we shall see that the top 10 eigenvectors explain about 85% of the square magnitude of the matrix.

<p><img src="hw5_figures/SVD.png" style="height:300px; width:450px"></p>

### Reordering matrix using eigen-vectors

First, we get the top 4 eigen-vectors.

<p><img src="hw5_figures/eigen_prcp.png"></p>

Then we reorder the rows and columns of the matrix using one of the eigenvectors. 
<p><img src="hw5_figures/reorder.png"></p>

After reordering, the grouping of the stations becomes more evident. For example, consider the upper left corner of the second matrix. The stations at positions 0-50 are clearly strongly correlated with each other. Even though there are some stations, in positions 15-22 or so, which are more related to each other than to the rest of this block.